In [2]:

%reload_ext autoreload
from pathlib import Path
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader
import numpy as np
# import matplotlib.pyplot as plt
import random
from einops import rearrange

from pandas import DataFrame, set_option

set_option('display.max_columns', None)
set_option('display.max_rows', None)


def nice_display(arr):
    if arr.ndim == 1:
        display(DataFrame(arr.reshape(1, -1)))
    elif arr.ndim == 2:
        display(DataFrame(arr))
    else:
        display(DataFrame(np.squeeze(arr)))

In [3]:

%autoreload 2
from dataset.crossView_UCLA_ske import NUCLA_CrossView
from modelZoo.sparseCoding import DyanEncoder as Dyan
# from torch.utils.tensorboard import SummaryWriter

In [4]:
setup = 'setup1'
dataType = '2D'
sampling = 'Single'
nClip = 1
maskType = 'None'
nw = 8
T = 36
gpu_id = 3

learning_rate = 1.0e-4

if gpu_id < torch.cuda.device_count():
    device = torch.device(('cuda:'+ str(gpu_id)) if torch.cuda.is_available() else 'cpu')
else: 
    device = 'cpu'
print(f'Pytorch Version: {torch.__version__} \nDevice {device} - {torch.cuda.get_device_name()}')

seed = 123

# Set seed for PyTorch (CPU)
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# Set seed for PyTorch (GPU)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If you use multiple GPUs

# Make PyTorch operations deterministic (slows down computation but improves reproducibility)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Pytorch Version: 1.9.1+cu111 
Device cuda:3 - NVIDIA GeForce RTX 2080 Ti


In [5]:
data_files_list = Path().home().joinpath('data', 'N-UCLA_MA_3D')
if data_files_list.exists():
        data_files_list = data_files_list.as_posix() + f"/data/CV/{setup}/"
else:
    raise FileExistsError

    
trainSet = NUCLA_CrossView(root_list=data_files_list, phase='train',
                            setup=setup, dataType=dataType,
                            sampling=sampling, nClip=nClip,
                            T=36, maskType=maskType) 
trainLoader = DataLoader(trainSet, shuffle=True,
                             batch_size=16, num_workers=nw)

testSet = NUCLA_CrossView(root_list=data_files_list, phase='test',
                            setup=setup, dataType=dataType,
                            sampling=sampling, nClip=nClip,
                            T=36, maskType=maskType) 
testLoader = DataLoader(trainSet, shuffle=False,
                             batch_size=1, num_workers=nw)

In [ ]:
train_views = [sample[0] for sample in trainSet.samples_list]
print(np.unique(train_views, return_counts=True))

(array(['view_1', 'view_2'], dtype='<U6'), array([516, 504]))


In [6]:
d_path = Path().cwd().joinpath('my_tests/dyan_test/NUCLA_CV_setup1_Single/DIR_D_wiRH/dir_d_100.pth')
poles = torch.load(d_path, map_location=device)['state_dict']
rr = poles['rr']
theta = poles['theta']

In [7]:

training_loss = []
C_list = []

dyan = Dyan(rr, theta, T, True, 0.1, device)
dyan.eval()
mseloss = nn.MSELoss()

optimizer = torch.optim.SGD(filter(lambda x: x.requires_grad, dyan.parameters()), 
                            lr=learning_rate, weight_decay=0.001, momentum=0.9)

for i, sample in enumerate(testLoader):
    # setup batches
    # skeletons = sample['input_skeletons']['normSkeleton'].float().to(device)
    # input_skeletons = rearrange(skeletons, 'n c t j d -> (n c) t (j d)')
    
    skeletons = sample['input_skeletons']['normSkeleton'].float().to(device)
    input_skeletons = rearrange(skeletons, 'n c t j d -> (n c) t (j d)')
            
    C_pred, _, pred_skeletons = dyan(input_skeletons)
    training_loss.append(mseloss(pred_skeletons, input_skeletons).detach())
    C_list.append(C_pred.detach())
    # if i == 0:
    #     break

In [70]:
nice_display(torch.count_nonzero(C_list[0], dim=1).cpu())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,26,21,35,25,31,24,11,28,19,43,50,26,54,41,60,43,39,38,26,29,32,54,37,56,51,36,56,57,57,67,29,17,37,18,28,19,55,27,66,68,65,67,56,62,37,57,34,61,41,57
1,47,20,59,22,51,21,49,46,31,49,57,23,62,35,55,46,45,43,43,42,52,51,52,56,58,38,51,48,67,57,42,0,54,21,0,0,68,16,46,60,57,59,70,59,40,58,36,65,57,59
2,29,21,39,20,28,21,22,31,32,38,50,22,61,34,63,42,51,43,34,42,43,50,55,55,59,40,67,47,61,54,30,19,40,21,34,18,55,4,60,56,59,59,60,55,57,57,52,55,55,60
3,28,17,45,20,38,19,54,41,46,54,48,17,55,25,50,34,53,34,56,32,53,46,50,55,55,37,57,48,63,53,24,20,33,21,29,0,37,15,48,56,56,59,65,54,54,54,53,59,64,76
4,41,17,47,27,34,19,14,32,25,31,55,22,60,28,60,40,45,29,28,31,39,41,49,55,55,28,58,47,55,50,38,21,49,28,43,18,56,10,58,51,60,50,52,53,50,61,48,58,50,59
5,58,25,49,18,40,17,27,23,41,37,61,29,65,37,64,40,50,36,35,32,44,47,46,55,51,33,57,48,46,56,51,23,72,27,50,23,0,0,61,58,61,67,47,53,41,55,36,55,47,54
6,28,13,48,17,53,19,66,35,56,41,44,22,41,32,37,39,55,34,61,31,59,47,60,55,49,36,50,50,45,55,37,0,36,15,29,0,40,13,39,62,38,66,48,56,52,54,52,54,56,55
7,45,14,42,20,29,16,16,25,33,27,48,20,57,32,54,39,38,25,33,31,35,49,41,56,48,29,56,48,53,58,39,12,53,10,38,4,74,9,72,59,69,72,54,58,48,56,48,54,43,57
8,35,27,44,20,50,20,53,33,68,47,37,16,30,29,24,36,48,36,52,33,54,48,51,58,44,35,38,48,50,58,0,0,35,23,63,5,37,15,44,61,44,62,50,59,52,61,73,89,55,56
9,53,18,37,16,29,14,17,25,12,33,50,18,55,31,61,36,29,38,31,41,37,51,49,55,55,34,52,50,50,57,48,14,52,16,35,13,0,0,63,52,63,54,48,57,55,55,52,56,49,56
